In [1]:
import pandas as pd

df = pd.read_csv("primary metadata.csv")

In [2]:
# Drop unnecessary columns
columns_to_drop = ['Target', 'Indicator Number', 'Metadata']
df = df.drop(columns=columns_to_drop, axis='columns')

In [3]:
# Pivoting the Indicator name column into multiple columns
df_pivoted = df.pivot_table(index=["Region", "Country", "Year"], columns="Indicator Name", values="Value").reset_index()

In [4]:
df.columns

Index(['Region', 'Country', 'Indicator Name', 'Year', 'Value'], dtype='object')

In [5]:
# Extracting gender information and remove it from the column names
df_pivoted.columns = df_pivoted.columns.str.replace(r'\s+\(%\w+\)$', '')
df_pivoted.columns.name = None

# Melting the pivoted DataFrame to reshape it
df_melted = df_pivoted.melt(id_vars=["Region", "Country", "Year"], var_name="Indicator Name", value_name="Value")

# Spliting the "Indicator Name" column into "Indicator" and "Gender"
df_melted[['Indicator', 'Gender']] = df_melted['Indicator Name'].str.extract(r'(.*)\s+(male|female)')

# Drop the "Indicator Name" column
df_melted = df_melted.drop(columns=["Indicator Name"])

# Pivot the melted DataFrame to create separate columns for each indicator and gender
df_final = df_melted.pivot_table(index=["Region", "Country", "Year", "Gender"], columns="Indicator", values="Value").reset_index()

column_order = ['Region', 'Country', 'Year', 'Gender'] + list(df_final.columns[4:])
df_final = df_final[column_order]


df_final.to_csv("KubeFlow Primary.csv")

In [6]:
df_final.columns

Index(['Region', 'Country', 'Year', 'Gender',
       'Completion rate, primary education,',
       'Gross enrolment ratio, early childhood education,',
       'Gross enrolment ratio, early childhood educational development programmes,',
       'Gross intake ratio to the last grade of primary education,',
       'Literacy rate, population 25-64 years,',
       'Percentage of children under 5 years experiencing positive and stimulating home learning environments,',
       'Percentage of students at the end of primary education who have their first or home language as language of instruction,',
       'Percentage of teachers in primary education who received in-service training in the last 12 months by type of training,',
       'Teacher attrition rate from primary education,'],
      dtype='object', name='Indicator')

In [7]:
df_final.shape

(4399, 13)

In [8]:
# Drop any duplicate rows from the merged dataframe
df_final = df_final.drop_duplicates()
df_final.shape

(4399, 13)

In [9]:
df_Addcol = pd.read_csv("primary metadata.csv")

columns_to_drop = ['Target', 'Indicator Number', 'Metadata']
df_Addcol = df_Addcol.drop(columns=columns_to_drop, axis='columns')

# Pivot the Indicator name column into multiple columns
df_Addcol = df_Addcol.pivot_table(index=["Region", "Country", "Year"], columns="Indicator Name", values="Value")

df_Addcol.to_csv("try1.csv")

In [10]:
# first dataset
df1 = pd.read_csv("KubeFlow Primary.csv")

# second dataset 
df2 = pd.read_csv("try1.csv")

# Selecting the columns from the second dataset that to add to the first one
cols = ["Existence of funding mechanisms to reallocate education resources to disadvantaged populations", "Expenditure on education as a percentage of total government expenditure (%)", "Government expenditure on education as a percentage of GDP (%)"]

# Merging the two dataframes on country and year, keeping only the selected columns from the second one
df_merged = pd.merge(df1, df2[["Country", "Year"] + cols], on=["Country", "Year"], how="left")

# Drop duplicate rows from the merged dataframe
df_merged = df_merged.drop_duplicates()



In [11]:
df_merged.shape

(4399, 17)

# Removing duplicate regions
As you can see below the region - Central Asia & Southern Asia are created separately but we a region Central and Southern Asia which has countries present in both/ so we will drop the separately created regions 

In [12]:

country_counts = df_merged["Country"].value_counts()

country = "India"

print("{} appears {} times in the first dataset.".format(country, country_counts[country]))

India appears 44 times in the first dataset.


In [13]:

regions = df_merged["Region"].unique()

print("The unique regions in the dataset are:")
print(regions)

The unique regions in the dataset are:
['SDG: Central Asia' 'SDG: Central and Southern Asia'
 'SDG: Eastern and South-Eastern Asia' 'SDG: Europe and Northern America'
 'SDG: Latin America and the Caribbean'
 'SDG: Northern Africa and Western Asia' 'SDG: Oceania'
 'SDG: Southern Asia' 'SDG: Sub-Saharan Africa']


In [14]:
# Grouping the dataframe by Region and get the unique countries in each group
country_groups = df_merged.groupby("Region")["Country"].unique()

print("The unique countries in each region are:")
print(country_groups)

The unique countries in each region are:
Region
SDG: Central Asia                        [Kazakhstan, Kyrgyzstan, Tajikistan, Turkmenis...
SDG: Central and Southern Asia           [Kazakhstan, Kyrgyzstan, Tajikistan, Turkmenis...
SDG: Eastern and South-Eastern Asia      [Brunei Darussalam, Cambodia, China, China, Ho...
SDG: Europe and Northern America         [Albania, Andorra, Austria, Belarus, Belgium, ...
SDG: Latin America and the Caribbean     [Anguilla, Antigua and Barbuda, Argentina, Aru...
SDG: Northern Africa and Western Asia    [Algeria, Armenia, Azerbaijan, Bahrain, Cyprus...
SDG: Oceania                             [Australia, Cook Islands, Fiji, Kiribati, Mars...
SDG: Southern Asia                       [Afghanistan, Bangladesh, Bhutan, India, Iran ...
SDG: Sub-Saharan Africa                  [Angola, Benin, Botswana, Burkina Faso, Burund...
Name: Country, dtype: object


In [15]:
# Drop the rows with the regions 'SDG: Central Asia' and 'SDG: Southern Asia'
df_merged = df_merged.drop(df_merged[(df_merged["Region"] == "SDG: Central Asia") | (df_merged["Region"] == "SDG: Southern Asia")].index)

In [16]:

country_counts = df_merged["Country"].value_counts()

country = "India"

print("{} appears {} times in the first dataset.".format(country, country_counts[country]))

India appears 22 times in the first dataset.


In [17]:
df_merged.shape

(4105, 17)

# Keeping only the devloping countries in the dataset

In [18]:
devloping_countries = ['Afghanistan',
'Albania',
'Algeria',
'American Samoa',
'Angola',
'Antigua and Barbuda',
'Argentina',
'Armenia',
'Azerbaijan',
'Bangladesh',
'Belarus',
'Belize',
'Benin',
'Bhutan',
'Bolivia',
'Bosnia and Herzegovina',
'Botswana',
'Brazil',
'Bulgaria',
'Burkina Faso',
'Burundi',
'Cambodia',
'Cameroon',
'Cape Verde',
'Central African Republic',
'Chad',
'Chile',
'China',
'Colombia',
'Comoros',
'Congo, Dem. Rep',
'Congo, Rep.',
'Costa Rica',
'Côte d’Ivoire',
'Cuba',
'Djibouti',
'Dominica',
'Dominican Republic',
'Ecuador',
'Egypt, Arab Rep.',
'El Salvador',
'Eritrea',
'Ethiopia',
'Fiji',
'Gabon',
'Gambia',
'Georgia',
'Ghana',
'Grenada',
'Guatemala',
'Guinea',
'Guinea-Bisau',
'Guyana',
'Haiti',
'Honduras',
'India',
'Indonesia',
'Iran, Islamic Rep.',
'Iraq',
'Jamaica',
'Jordan',
'Kazakhstan',
'Kenya',
'Kiribati',
'Korea, Dem Rep.',
'Kosovo',
'Kyrgyz Republic',
'Lao PDR',
'Latvia',
'Lebanon',
'Lesotho',
'Liberia',
'Libya',
'Lithuania',
'Macedonia, FYR',
'Madagascar',
'Malawi',
'Malaysia',
'Maldives',
'Mali',
'Marshall Islands',
'Mauritania',
'Mauritius',
'Mayotte',
'Mexico',
'Micronesia, Fed. Sts.',
'Moldova',
'Mongolia',
'Montenegro',
'Morocco',
'Mozambique',
'Myanmar',
'Namibia',
'Nepal',
'Nicaragua',
'Niger',
'Nigeria',
'Pakistan',
'Palau',
'Panama',
'Papua New Guinea',
'Paraguay',
'Peru',
'Philippines',
'Romania',
'Russian Federation',
'Rwanda',
'Samoa',
'São Tomé and Principe',
'Senegal',
'Serbia',
'Seychelles',
'Sierra Leone',
'Solomon Islands',
'Somalia',
'South Africa',
'Sri Lanka',
'St. Kitts and Nevis',
'St. Lucia',
'St. Vincent and the Grenadines',
'Sudan',
'Suriname',
'Swaziland',
'Syrian Arab Republic',
'Tajikistan',
'Tanzania',
'Thailand',
'Timor-Leste',
'Togo',
'Tonga',
'Tunisia',
'Turkey',
'Turkmenistan',
'Tuvalu',
'Uganda',
'Ukraine',
'Uruguay',
'Uzbekistan',
'Vanuatu',
'Venezuela',
'Vietnam',
'West Bank and Gaza',
'Yemen, Rep.',
'Zambia',
'Zimbabwe']


In [19]:
len(devloping_countries)

145

In [20]:
filtered_df = df_merged[df_merged['Country'].isin(devloping_countries)]

In [21]:
filtered_df.shape

(2353, 17)

In [22]:
# Saving the dataframe as a new CSV file
filtered_df.to_csv("Primary final.csv", index=False)